In [103]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore') 

In [104]:
df = pd.read_csv("./데이터자료/9-2. Std 데이터셋.csv")
feature = pd.read_csv("./데이터자료/8-2. Std feature.csv")
df

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,산업명,매출액증가율,유형자산증가율,재고자산증가율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,label,회계년
0,(주)CMG제약,58820,2011-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,1.186494,-0.286555,-0.242712,...,-0.517775,-0.268715,-0.204982,-0.097181,0.001823,0.167352,-0.034998,-0.128958,0.0,2011
1,(주)CMG제약,58820,2012-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,-0.465320,-0.266139,-0.145700,...,-0.570848,-0.965957,-0.239707,-0.097246,-0.455154,7.331845,-8.395971,-8.842764,0.0,2012
2,(주)CMG제약,58820,2013-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.399443,-0.034331,0.368591,...,-0.459402,-0.280668,-0.160693,-0.097092,0.377872,-0.016097,0.127964,0.136844,0.0,2013
3,(주)CMG제약,58820,2014-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.042989,0.633335,-0.160104,...,-0.458301,-0.300589,-0.202938,-0.097179,0.294901,0.063259,0.057470,0.125748,0.0,2014
4,(주)CMG제약,58820,2015-12-01,2001-08-31,NaN,5,의료용 물질 및 의약품 제조업,0.229752,-0.280476,-0.110645,...,-0.503803,-0.558426,-0.217252,-0.097201,-0.090083,0.704439,-0.512105,-0.358349,0.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9273,휴마시스(주),205470,2020-12-01,2014-12-01,NaN,5,의료용 물질 및 의약품 제조업,5.662930,1.347455,2.510801,...,-0.151711,1.140567,-0.125778,-0.096931,0.930074,-0.576239,0.625551,0.569922,0.0,2020
9274,휴젤(주),145020,2019-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,0.059296,-0.046260,0.113775,...,1.299954,-0.272699,-0.062404,-0.097006,0.914756,-0.434150,0.499330,0.360994,0.0,2019
9275,휴젤(주),145020,2020-12-01,2015-12-24,NaN,5,의료용 물질 및 의약품 제조업,-0.170016,-0.100971,-0.437478,...,1.349171,-0.301727,-0.058107,-0.096969,0.907353,-0.415350,0.482630,0.330828,0.0,2020
9276,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,5,도매 및 상품 중개업,-0.488761,-0.299172,0.793390,...,-0.392823,-0.500939,-0.145811,-0.095817,-0.354313,-0.106535,0.208302,0.265758,0.0,2019


In [105]:
train = df[df["회계년"] <= 2018].reset_index(drop='index')
test  = df[df["회계년"] > 2018].reset_index(drop='index')

In [106]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test.drop("label",axis=1)
y_test = test["label"]

In [107]:
mda_feature=feature["mda_feature"].tolist()
logit_feature=feature["logit_feature"].tolist()
lasso_feature=feature["lasso_feature"].tolist()
ela_feature=feature["ela_feature"].tolist()
same_feature=feature["same_feature"].tolist()

In [108]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))
   

In [109]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")



In [110]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

# 1. mda_feature

In [111]:
mda_x_train = x_train[mda_feature].reset_index(drop='index')
mda_x_test = x_test[mda_feature].reset_index(drop='index')
mda_y_train = train["label"]
mda_y_test = test["label"]

In [112]:
logit_x_train = x_train[logit_feature].reset_index(drop='index')
logit_x_test = x_test[logit_feature].reset_index(drop='index')
logit_y_train = train["label"]
logit_y_test = test["label"]

In [113]:
lasso_x_train = x_train[lasso_feature].reset_index(drop='index')
lasso_x_test = x_test[lasso_feature].reset_index(drop='index')
lasso_y_train = train["label"]
lasso_y_test = test["label"]

In [114]:
ela_x_train = x_train[ela_feature].reset_index(drop='index')
ela_x_test = x_test[ela_feature].reset_index(drop='index')
ela_y_train = train["label"]
ela_y_test = test["label"]

In [115]:
same_x_train = x_train[same_feature].reset_index(drop='index')
same_x_test = x_test[same_feature].reset_index(drop='index')
same_y_train = train["label"]
same_y_test = test["label"]

In [116]:
fea_x_train = [mda_x_train, logit_x_train, lasso_x_train, ela_x_train, same_x_train]
fea_y_train = [mda_y_train, logit_y_train, lasso_y_train, ela_y_train, same_y_train]
fea_x_test = [mda_x_test, logit_x_test, lasso_x_test, ela_x_test, same_x_test]
fea_y_test = [mda_y_test, logit_y_test, lasso_y_test, ela_y_test, same_y_test]
fea = ["mda","logit","lasso","ela","same"]

In [117]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lda_clf.fit(x_train, y_train)
    lda_pred = lda_clf.predict(x_test)
    lda_pred_proba = lda_clf.predict_proba(x_test)
    print("---------------------------------------------------------")
    print(feature,"lda") 
    get_clf_eval(y_test, lda_pred)
    get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)
    

---------------------------------------------------------
mda lda
오차행렬
[[2252   63]
 [  32   18]]
정확도: 0.9598, 정밀도 : 0.2222, 재현율:0.3600,F1 스코어:0.2748
ROC 스코어: 0.6664, PR score : 0.0935
임곗값: 0.1
오차행렬
[[2171  144]
 [  26   24]]
정확도: 0.9281, 정밀도 : 0.1429, 재현율:0.4800,F1 스코어:0.2202
ROC 스코어: 0.7089, PR score : 0.0796
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2210  105]
 [  27   23]]
정확도: 0.9442, 정밀도 : 0.1797, 재현율:0.4600,F1 스코어:0.2584
ROC 스코어: 0.7073, PR score : 0.0941
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2229   86]
 [  27   23]]
정확도: 0.9522, 정밀도 : 0.2110, 재현율:0.4600,F1 스코어:0.2893
ROC 스코어: 0.7114, PR score : 0.1085
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2243   72]
 [  29   21]]
정확도: 0.9573, 정밀도 : 0.2258, 재현율:0.4200,F1 스코어:0.2937
ROC 스코어: 0.6944, PR score : 0.1071
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2252   63]
 [  32   18]]
정확도: 0.9598, 정밀도 : 0.2222, 재현율

In [118]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    dt_clf.fit(x_train, y_train)
    dt_pred = dt_clf.predict(x_test)
    dt_pred_proba = dt_clf.predict_proba(x_test)   
    print(feature,"디시전트리") 
    get_clf_eval(y_test, dt_pred)
    get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 디시전트리
오차행렬
[[2267   48]
 [  41    9]]
정확도: 0.9624, 정밀도 : 0.1579, 재현율:0.1800,F1 스코어:0.1682
ROC 스코어: 0.5796, PR score : 0.0458
임곗값: 0.1
오차행렬
[[2267   48]
 [  41    9]]
정확도: 0.9624, 정밀도 : 0.1579, 재현율:0.1800,F1 스코어:0.1682
ROC 스코어: 0.5796, PR score : 0.0458
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2267   48]
 [  41    9]]
정확도: 0.9624, 정밀도 : 0.1579, 재현율:0.1800,F1 스코어:0.1682
ROC 스코어: 0.5796, PR score : 0.0458
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2267   48]
 [  41    9]]
정확도: 0.9624, 정밀도 : 0.1579, 재현율:0.1800,F1 스코어:0.1682
ROC 스코어: 0.5796, PR score : 0.0458
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2267   48]
 [  41    9]]
정확도: 0.9624, 정밀도 : 0.1579, 재현율:0.1800,F1 스코어:0.1682
ROC 스코어: 0.5796, PR score : 0.0458
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2267   48]
 [  41    9]]
정확도: 0.9624, 정밀도 : 0.1579, 재현율:0.1800,F1 스코어:0.1682
ROC 스코어: 0.5796, PR score : 0.0458

In [119]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    # key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
    param_grid = {
      'criterion': ['entropy', 'gini'],
               'max_depth': [2,4,6,8],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [4,6,8,10],
               'min_samples_split': [4,6,8,10]}
    grid_search = GridSearchCV(tree
                            , param_grid = param_grid
                            , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                            , n_jobs = -1)  #-1: 모든 코어 사용

    grid_search.fit(x_train, y_train)

    # 최선의 결과 출력
    print('피처:{2} 최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_, feature))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
    dt_clf_best.fit(x_train, y_train)
    dt_best_pred = dt_clf_best.predict(x_test)
    dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

    print(feature,"디시전트리 파라미터적용") 
    get_clf_eval(y_test, dt_best_pred)
    get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


KeyboardInterrupt: 

In [ ]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    rf_clf.fit(x_train, y_train)
    rf_pred = rf_clf.predict(x_test)
    rf_pred_proba = rf_clf.predict_proba(x_test)
    print(feature,"랜덤포레스트") 
    get_clf_eval(y_test, rf_pred)
    get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)
    


mda 랜덤포레스트
오차행렬
[[2308    7]
 [  41    9]]
정확도: 0.9797, 정밀도 : 0.5625, 재현율:0.1800,F1 스코어:0.2727
ROC 스코어: 0.5885, PR score : 0.1186
---------------------------------------------------------
임곗값: 0.1
오차행렬
[[2212  103]
 [  26   24]]
정확도: 0.9455, 정밀도 : 0.1890, 재현율:0.4800,F1 스코어:0.2712
ROC 스코어: 0.7178, PR score : 0.1017
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2280   35]
 [  35   15]]
정확도: 0.9704, 정밀도 : 0.3000, 재현율:0.3000,F1 스코어:0.3000
ROC 스코어: 0.6424, PR score : 0.1048
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2294   21]
 [  39   11]]
정확도: 0.9746, 정밀도 : 0.3438, 재현율:0.2200,F1 스코어:0.2683
ROC 스코어: 0.6055, PR score : 0.0921
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2301   14]
 [  41    9]]
정확도: 0.9767, 정밀도 : 0.3913, 재현율:0.1800,F

In [ ]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    grid_search = {'criterion': ['entropy', 'gini'],
                'max_depth': [2,4,6,8],
                'max_features': ['auto', 'sqrt'],
                'min_samples_leaf': [4,6,8,10],
                'min_samples_split': [4,6,8,10],
                'n_estimators': [20,40,60,80,100]}

    model = RandomForestClassifier(random_state=52)
    grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                                cv = 5,verbose=0, n_jobs = -1)
    grid_search.fit(x_train,y_train)

    # 최선의 결과 출력
    print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
    rf_clf_best.fit(x_train, y_train)
    rf_best_pred = rf_clf_best.predict(x_test)
    rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

    print(feature,"랜덤포레스트 파라미터적용") 
    get_clf_eval(y_test, rf_best_pred)
    get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'entropy', 'max_depth': 8, 'max_features': 'auto', 'min_samples_leaf': 10, 'min_samples_split': 4, 'n_estimators': 20}, 최적 평균 정확도 :0.984
mda 디시전트리 파라미터적용
오차행렬
[[2310    5]
 [  43    7]]
정확도: 0.9797, 정밀도 : 0.5833, 재현율:0.1400,F1 스코어:0.2258
ROC 스코어: 0.5689, PR score : 0.0998
---------------------------------------------------------
임곗값: 0.1
오차행렬
[[2220   95]
 [  25   25]]
정확도: 0.9493, 정밀도 : 0.2083, 재현율:0.5000,F1 스코어:0.2941
ROC 스코어: 0.7295, PR score : 0.1147
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2281   34]
 [  37   13]]
정확도: 0.9700, 정밀도 : 0.2766, 재현율:0.2600,F1 스코어:0.2680
ROC 스코어: 0.6227, PR score : 0.0876
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2303   12]
 [  39   11]]
정확도: 0.9784, 정밀도 : 0.4783, 재현율:0.2200,F1 스코어:0.3014
ROC 스코어: 0.6074, PR score : 0.1217
--------------------------

KeyboardInterrupt: 

In [ ]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    lr_clf.fit(x_train, y_train)
    lr_pred = lr_clf.predict(x_test)
    lr_pred_proba = lr_clf.predict_proba(x_test)
    print(feature,"로지스틱회귀") 
    get_clf_eval(y_test, lr_pred)
    get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

mda 로지스틱회귀
오차행렬
[[2309    6]
 [  46    4]]
정확도: 0.9780, 정밀도 : 0.4000, 재현율:0.0800,F1 스코어:0.1333
ROC 스코어: 0.5387, PR score : 0.0515
---------------------------------------------------------
임곗값: 0.1
오차행렬
[[2253   62]
 [  26   24]]
정확도: 0.9628, 정밀도 : 0.2791, 재현율:0.4800,F1 스코어:0.3529
ROC 스코어: 0.7266, PR score : 0.1449
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2292   23]
 [  33   17]]
정확도: 0.9763, 정밀도 : 0.4250, 재현율:0.3400,F1 스코어:0.3778
ROC 스코어: 0.6650, PR score : 0.1585
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2302   13]
 [  39   11]]
정확도: 0.9780, 정밀도 : 0.4583, 재현율:0.2200,F1 스코어:0.2973
ROC 스코어: 0.6072, PR score : 0.1173
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2307    8]
 [  44    6]]
정확도: 0.9780, 정밀도 : 0.4286, 재현율:0.1200,F

In [ ]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {'penalty': ['l2','l1','elasticnet','None'],
            'C':[0.01,0.1,1,3,5,10]}

    grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
    grid_search.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

    a = grid_search.best_params_
    keys=list(a.keys())
    values = list(a.values())

    lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
    lr_clf_best.fit(x_train, y_train)
    lr_best_pred = lr_clf_best.predict(x_test)
    lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

    print(feature,"로지스틱회귀 파라미터적용") 
    get_clf_eval(y_test, lr_best_pred)
    get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


In [ ]:
# 1-5 KNN 
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_neighbors=5)
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    knn_clf.fit(x_train, y_train)
    knn_pred = knn_clf.predict(x_test)
    knn_pred_proba = knn_clf.predict_proba(x_test)
    print(feature,"KNN") 
    get_clf_eval(y_test, knn_pred)
    get_eval_by_threshold(y_test , knn_pred_proba[:,1].reshape(-1,1), thresholds)



mda KNN
오차행렬
[[2312    3]
 [  45    5]]
정확도: 0.9797, 정밀도 : 0.6250, 재현율:0.1000,F1 스코어:0.1724
ROC 스코어: 0.5494, PR score : 0.0815
---------------------------------------------------------
임곗값: 0.1
오차행렬
[[2192  123]
 [  29   21]]
정확도: 0.9357, 정밀도 : 0.1458, 재현율:0.4200,F1 스코어:0.2165
ROC 스코어: 0.6834, PR score : 0.0735
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2300   15]
 [  40   10]]
정확도: 0.9767, 정밀도 : 0.4000, 재현율:0.2000,F1 스코어:0.2667
ROC 스코어: 0.5968, PR score : 0.0969
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2300   15]
 [  40   10]]
정확도: 0.9767, 정밀도 : 0.4000, 재현율:0.2000,F1 스코어:0.2667
ROC 스코어: 0.5968, PR score : 0.0969
---------------------------------------------------------
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2312    3]
 [  45    5]]
정확도: 0.9797, 정밀도 : 0.6250, 재현율:0.1000,F1 스

In [ ]:
# knn 파라미터 구하기
from sklearn.metrics import accuracy_score
acc = []
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    for k in range(1,20):
        knn_clf = KNeighborsClassifier(n_neighbors=k)
        knn_clf.fit(x_train, y_train)
        knn_pred = knn_clf.predict(x_test)
        knn_score=accuracy_score(y_test,knn_pred)

        acc.append(knn_score)
    j = acc.index(max(acc))+1
    print("최적거리")
    print(j)

    knn_clf_best =KNeighborsClassifier(n_neighbors=j)
    knn_clf_best.fit(x_train, y_train)
    knn_best_pred = knn_clf_best.predict(x_test)
    knn_best_pred_proba = knn_clf_best.predict_proba(x_test)   

    print(feature,"KNN 파라미터적용") 
    get_clf_eval(y_test, knn_best_pred)
    get_eval_by_threshold(y_test ,knn_best_pred_proba[:,1].reshape(-1,1), thresholds)      


In [ ]:
# 1-5 xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb ## XGBoost 불러오기
from xgboost import plot_importance 

xgb_wrapper = XGBClassifier()
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):

    xgb_wrapper.fit(x_train, y_train, verbose = True)
    xgb_wrapper.fit(x_train, y_train)
    xgb_pred = xgb_wrapper.predict(x_test)
    xgb_pred_proba = xgb_wrapper.predict_proba(x_test)
    print(feature,"XGB") 
    get_clf_eval(y_test, xgb_pred)
    get_eval_by_threshold(y_test , xgb_pred_proba[:,1].reshape(-1,1), thresholds)

mda XGB
오차행렬
[[2302   13]
 [  40   10]]
정확도: 0.9776, 정밀도 : 0.4348, 재현율:0.2000,F1 스코어:0.2740
ROC 스코어: 0.5972, PR score : 0.1039
임곗값: 0.1
오차행렬
[[2277   38]
 [  38   12]]
정확도: 0.9679, 정밀도 : 0.2400, 재현율:0.2400,F1 스코어:0.2400
ROC 스코어: 0.6118, PR score : 0.0737
임곗값: 0.2
오차행렬
[[2292   23]
 [  39   11]]
정확도: 0.9738, 정밀도 : 0.3235, 재현율:0.2200,F1 스코어:0.2619
ROC 스코어: 0.6050, PR score : 0.0877
임곗값: 0.3
오차행렬
[[2298   17]
 [  39   11]]
정확도: 0.9763, 정밀도 : 0.3929, 재현율:0.2200,F1 스코어:0.2821
ROC 스코어: 0.6063, PR score : 0.1029
임곗값: 0.4
오차행렬
[[2299   16]
 [  39   11]]
정확도: 0.9767, 정밀도 : 0.4074, 재현율:0.2200,F1 스코어:0.2857
ROC 스코어: 0.6065, PR score : 0.1061
임곗값: 0.5
오차행렬
[[2302   13]
 [  40   10]]
정확도: 0.9776, 정밀도 : 0.4348, 재현율:0.2000,F1 스코어:0.2740
ROC 스코어: 0.5972, PR score : 0.1039
임곗값: 0.6
오차행렬
[[2305   10]
 [  41    9]]
정확도: 0.9784, 정밀도 : 0.4737, 재현율:0.1800,F1 스코어:0.2609
ROC 스코어: 0.5878, PR score : 0.1026
임곗값: 0.7
오차행렬
[[2308    7]
 [  42    8]]
정확도: 0.9793, 정밀도 : 0.5333, 재현율:0.1600,F1 스코어:0.2462
ROC 스코어: 0.5

In [ ]:
#xgb 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

xgb_wrapper
for x_train,y_train,x_test,y_test,feature in zip(fea_x_train,fea_y_train,fea_x_test, fea_y_test, fea):
    params = {
            'n_estimators':[40,43,46,50],
            'learning_rate':[0.03,0.05,0.04],
            'max_depth' : [0.8,1,2,3,4]
            }

    grid_clf = GridSearchCV(xgb_wrapper, param_grid=params, scoring='accuracy', cv=5)
    grid_clf.fit(x_train,y_train)
    print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_clf.best_params_, grid_clf.best_score_))

    a = grid_clf.best_params_
    keys=list(a.keys())
    values = list(a.values())
    xgb_clf_best = XGBClassifier( learning_rate=values[0],max_depth=values[1],n_estimators= values[2])
    xgb_clf_best.fit(x_train, y_train)
    xgb_best_pred = xgb_clf_best.predict(x_test)
    xgb_best_pred_proba = xgb_clf_best.predict_proba(x_test)   

    print(feature,"XGB 파라미터적용") 
    get_clf_eval(y_test, xgb_best_pred)
    get_eval_by_threshold(y_test ,xgb_best_pred_proba[:,1].reshape(-1,1), thresholds)